# Workshop: Workspace Setup, Data Import & Exploration

**Cel szkoleniowy:** Praktyczne opanowanie konfiguracji workspace, importowania danych z różnych formatów oraz podstawowych operacji eksploracyjnych.

**Zakres tematyczny:**
- Konfiguracja workspace i klastrów
- Wczytywanie różnych formatów danych (CSV, JSON, Parquet)
- Podstawowa eksploracja danych
- Konstruowanie schematów manualnie
- Analiza braków danych i wartości unikalnych

**Czas trwania:** 90 minut

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 1 - Fundamentals & Exploration
- **Typ notebooka**: Workshop
- **Wymagania techniczne**:
 - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
 - Unity Catalog włączony
 - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
 - Klaster: Standard z minimum 2 workers

## Wprowadzenie do warsztatu

W tym warsztacie będziesz pracować z rzeczywistymi danymi KION:
- **Customers** (customers.csv) - dane klientów
- **Orders** (orders_batch.json) - zamówienia
- **Products** (products.parquet) - produkty

### Zadania do wykonania:
1. Konfiguracja środowiska i zmiennych
2. Import danych z CSV, JSON, Parquet
3. Konstrukcja schematów manualnie
4. Eksploracja danych: statystyki, braki, unikalne wartości
5. Analiza jakości danych

### Kryteria sukcesu:
- Wszystkie 3 datasety poprawnie wczytane
- Schematy zdefiniowane manualnie i zastosowane
- Kompletna analiza eksploracyjna przeprowadzona
- Zidentyfikowane problemy jakości danych

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie podstaw pracy z danymi w Databricks Lakehouse

**Podstawowe pojęcia:**
- **Workspace**: Środowisko Databricks zawierające notebooks, klastry i dane
- **Cluster**: Zbiór maszyn wirtualnych przetwarzających dane
- **DataFrame**: Rozproszona kolekcja danych zorganizowana w kolumny
- **Schema**: Struktura danych definiująca nazwy kolumn i typy danych
- **Format danych**: CSV (text), JSON (semi-structured), Parquet (columnar binary)

**Dlaczego to ważne?**
Poprawne wczytanie i eksploracja danych to fundament każdego pipeline'u ETL. Zrozumienie schematów, formatów i metod eksploracji pozwala na wczesne wykrycie problemów jakości danych.

## Inicjalizacja środowiska

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../../00_setup

## Konfiguracja

Zdefiniuj zmienne specyficzne dla warsztatu:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Ścieżki do plików danych (już zdefiniowane w 00_setup)
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

**Kontekst konfiguracji**

Ścieżki do rzeczywistych plików danych zostały skonfigurowane:
- **Customers CSV**: customers.csv (~10,000 rekordów klientów)
- **Orders JSON**: orders_batch.json (~100,000 zamówień) 
- **Products Parquet**: products.parquet (katalog produktów)

**Rzeczywista struktura danych:**
- **Customers**: `customer_id`, `first_name`, `last_name`, `email`, `phone`, `city`, `state`, `country`, `registration_date`, `customer_segment`
- **Orders**: `order_id`, `customer_id`, `product_id`, `store_id`, `order_datetime`, `quantity`, `unit_price`, `discount_percent`, `total_amount`, `payment_method`

---

## Zadanie 1: Import danych CSV (20 min)

### Cel:
Wczytaj dane klientów z pliku CSV, najpierw z automatycznym wykrywaniem schematu, potem z ręcznie zdefiniowanym schematem.

### Instrukcje:
1. Wczytaj `customers.csv` z opcją `inferSchema=True`
2. Wyświetl schemat i 5 pierwszych rekordów
3. Policz liczbę rekordów
4. Zdefiniuj schemat manualnie (StructType)
5. Wczytaj ponownie używając ręcznego schematu
6. Porównaj schematy

### Oczekiwany rezultat:
- DataFrame z danymi klientów
- Schemat zawierający kolumny: customer_id (int), first_name (string), last_name (string), email (string), city (string), country (string), registration_date (timestamp)

### Wskazówki:
- Użyj `spark.read.format("csv").option("header", "true")`
- Do ręcznego schematu użyj: `StructType`, `StructField`, `IntegerType`, `StringType`, `TimestampType`

In [0]:
# Wczytanie pliku CSV z klientami (automatyczne wykrywanie schematu)
customers_df = (
 spark.read
 .format("csv")
 .option("header", "true")
 .option("inferSchema", "true")
 .load(CUSTOMERS_CSV)
)

# Wyświetl schemat i przykładowe dane
customers_df.printSchema()
display(customers_df.limit(5))

**Analiza danych customers po wczytaniu**

Spark automatycznie wykrył schemat. Zauważ rzeczywiste kolumny w pliku CSV:
- **Identyfikacja**: `customer_id` (CUST000001...)
- **Dane osobowe**: `first_name`, `last_name`, `email`, `phone`
- **Lokalizacja**: `city`, `state`, `country`
- **Metadane**: `registration_date`, `customer_segment`

**Następny krok**: Zdefiniuj schemat manualnie dla lepszej kontroli typów danych i wydajności.

In [0]:
# Zdefiniuj schemat manualnie na podstawie rzeczywistych danych
# Uzupełnij typy danych dla każdego pola
customers_schema = StructType([
 StructField("customer_id", ____, False), # StringType (CUST000001)
 StructField("first_name", ____, True), # StringType
 StructField("last_name", StringType(), True),
 StructField("email", ____, True), # StringType
 StructField("phone", StringType(), True),
 StructField("city", ____, True), # StringType
 StructField("state", StringType(), True),
 StructField("country", ____, True), # StringType
 StructField("registration_date", ____, True), # TimestampType
 StructField("customer_segment", ____, True) # StringType
])

# Wczytaj ponownie z ręcznym schematem
customers_df_manual = (
 spark.read
 .format("____") # csv
 .option("header", "____") # true
 .schema(____) # customers_schema
 .load(CUSTOMERS_CSV)
)

# Wyświetl schemat
print("[INFO] Schemat z ręczną definicją:")
customers_df_manual.printSchema()

# Wyświetl przykładowe dane
display(customers_df_manual.limit(5))

**Schemat customers zdefiniowany ręcznie**

 **Best practice**: Zawsze definiuj schemat manualnie zamiast używać `inferSchema=True`

**Korzyści ręcznego schematu:**
- **Wydajność**: Brak potrzeby skanowania całego pliku do określenia typów
- **Przewidywalność**: Kontrola nad typami danych (String vs Integer)
- **Bezpieczeństwo**: Walidacja zgodności danych ze schematem
- **Dokumentacja**: Schemat służy jako dokumentacja struktury danych

---

## Zadanie 2: Import danych JSON (15 min)

### Cel:
Wczytaj dane zamówień z pliku JSON i zdefiniuj schemat manualnie.

### Instrukcje:
1. Wczytaj `orders_batch.json` z `inferSchema=True`
2. Zbadaj strukturę danych (schemat, typy)
3. Zdefiniuj schemat manualnie
4. Wczytaj ponownie z ręcznym schematem

### Oczekiwany rezultat:
- DataFrame z zamówieniami
- Schemat: order_id (int), customer_id (int), order_date (timestamp), total_amount (double), status (string)

### Wskazówki:
- JSON nie wymaga opcji `header`
- Użyj `DoubleType` dla kwot pieniężnych

In [0]:
# Wczytanie pliku JSON z zamówieniami (automatyczne wykrywanie schematu)
orders_df = (
 spark.read
 .format("json")
 .option("multiLine", "true")
 .load(ORDERS_JSON)
)

display(orders_df.limit(5))

**Analiza danych orders po wczytaniu z JSON**

Zauważ rzeczywistą strukturę danych zamówień:
- **Identyfikatory**: `order_id`, `customer_id`, `product_id`, `store_id`
- **Szczegóły transakcji**: `order_datetime`, `quantity`, `unit_price`, `discount_percent`
- **Podsumowanie**: `total_amount`, `payment_method`

** Problemy jakości danych:**
- Niektóre rekordy mają `NULL` w `order_id` lub `order_datetime`
- Przyszłe daty w `order_datetime` (2026)
- Wymagane sprawdzenie spójności danych

In [0]:
# Zdefiniuj schemat orders na podstawie rzeczywistych danych
orders_schema = StructType([
 StructField("order_id", ____, True), # StringType (może być NULL)
 StructField("customer_id", ____, True), # StringType
 StructField("product_id", StringType(), True),
 StructField("store_id", ____, True), # StringType
 StructField("order_datetime", ____, True), # TimestampType (może być NULL)
 StructField("quantity", ____, True), # IntegerType
 StructField("unit_price", DoubleType(), True),
 StructField("discount_percent", ____, True), # IntegerType
 StructField("total_amount", ____, True), # DoubleType
 StructField("payment_method", ____, True) # StringType
])

# Wczytaj z ręcznym schematem
orders_df_manual = (
 spark.read
 .format("____") # json
 .option("multiLine", "____") # true
 .schema(____) # orders_schema
 .load(ORDERS_JSON)
)

orders_df_manual.printSchema()
display(orders_df_manual.limit(5))

---

## Zadanie 3: Import danych Parquet (10 min)

### Cel:
Wczytaj dane produktów z pliku Parquet (schemat jest wbudowany).

### Instrukcje:
1. Wczytaj `products.parquet`
2. Sprawdź schemat (Parquet zawiera schemat wbudowany)
3. Wyświetl dane
4. Policz rekordy

### Oczekiwany rezultat:
- DataFrame z produktami
- Schemat automatycznie załadowany z pliku Parquet

### Wskazówki:
- Parquet nie wymaga `inferSchema` ani ręcznego schematu

In [0]:
# Wczytaj products.parquet
products_df = (
 spark.read
 .format("____") # parquet
 .load(PRODUCTS_PARQUET)
)

# Wyświetl schemat (Parquet zawiera wbudowany schemat)
products_df.printSchema()

# Wyświetl dane i policz rekordy
display(products_df.limit(5))
product_count = products_df.count()

**Parquet - format z wbudowanym schematem**

 **Korzyści formatu Parquet:**
- **Schemat wbudowany**: Nie wymaga definiowania schematu manualnie
- **Kompresja kolumnowa**: Oszczędność miejsca i szybsze zapytania analityczne
- **Wydajność**: Najlepszy format do Big Data i analytics w lakehouse
- **Kompatybilność**: Uniwersalny standard dla systemów analitycznych

---

## Zapis danych do Delta Lake

### Cel:
Zapisz wczytane DataFrames do tabel Delta Lake dla dalszego użycia.

### Instrukcje:
1. Zapisz customers do tabeli `bronze.customers_workshop`
2. Zapisz orders do tabeli `bronze.orders_workshop`
3. Zapisz products do tabeli `bronze.products_workshop`

In [0]:
# Przykład: Zapis customers do Delta Lake
# Ta komórka jest już gotowa - przeanalizuj kod i uruchom ją

customers_table = f"{BRONZE_SCHEMA}.customers_workshop"

(
 customers_df_manual
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(customers_table)
)

# Sprawdź strukturę zapisanej tabeli
spark.sql(f"DESCRIBE TABLE {customers_table}").show(truncate=False)

**Zapis customers do Delta Lake wykonany**

 **Delta Lake - format ACID dla lakehouse:**
- **ACID transactions**: Atomowe operacje na danych
- **Schema evolution**: Możliwość zmiany schematu w czasie
- **Time travel**: Dostęp do historycznych wersji danych
- **Optimize & Vacuum**: Optymalizacja wydajności

Tabela zapisana: `{customers_table}`

In [0]:
# Zapisz orders do Delta Lake
orders_table = f"{BRONZE_SCHEMA}.orders_workshop"

(
 orders_df_manual
 .write
 .format("____") # delta
 .mode("____") # overwrite
 .option("overwriteSchema", "true")
 .saveAsTable(____) # orders_table
)

**Orders zapisane do Delta Lake**

Po uzupełnieniu braków powyżej, tabela orders zostanie zapisana w formacie Delta Lake. 
Zauważ użycie parametrów:
- **format("delta")**: Specyfikuje format Delta Lake
- **mode("overwrite")**: Zastępuje istniejące dane
- **overwriteSchema**: Pozwala na zmianę schematu tabeli

In [0]:
# Zapisz products do Delta Lake
products_table = f"{BRONZE_SCHEMA}.products_workshop"

(
 ____ # products_df
 .write
 .format("____") # delta
 .mode("____") # overwrite
 .option("overwriteSchema", "true")
 .saveAsTable(____) # products_table
)

**Products zapisane do Delta Lake**

Uzupełnij braki powyżej, aby zapisać tabelę produktów. Wszystkie trzy tabele (customers, orders, products) będą dostępne w schemacie bronze do dalszego przetwarzania.

---

## Zadanie 4: Eksploracja danych - Customers (15 min)

### Cel:
Przeprowadź szczegółową eksplorację danych klientów.

### Instrukcje:
1. Wyświetl listę kolumn i ich typy
2. Policz liczbę unikalnych klientów
3. Znajdź liczbę klientów według krajów
4. Sprawdź, czy są wartości NULL w kolumnach
5. Wygeneruj statystyki opisowe (`describe()`)

### Oczekiwany rezultat:
- Kompletna analiza eksploracyjna
- Zidentyfikowane braki danych
- Rozkład geograficzny klientów

### Wskazówki:
- Użyj: `columns`, `dtypes`, `count()`, `distinct()`, `groupBy()`, `describe()`
- Do sprawdzenia NULL: `.filter(col("column_name").isNull())`

In [0]:
# Podstawowe informacje o strukturze danych
customers_df_manual.columns
customers_df_manual.dtypes

# Policz unikalne wartości customer_id
unique_customers = customers_df_manual.select("____").distinct().count() # customer_id
total_customers = customers_df_manual.count()

# Wyświetl statystyki
display(spark.createDataFrame([
 (total_customers, unique_customers, total_customers - unique_customers)
], ["total_records", "unique_customers", "duplicates"]))

**Podstawowe statystyki customers**

Sprawdziliśmy:
- **Kolumny i typy**: Struktura danych oraz typy każdej kolumny
- **Duplikaty**: Czy `customer_id` jest unikalny (klucz główny)
- **Kompletność**: Całkowita liczba rekordów vs unikalne identyfikatory

**Następny krok**: Sprawdź rozkład geograficzny klientów

In [0]:
# Policz klientów według krajów
customers_by_country = (
 customers_df_manual
 .groupBy("____") # country
 .count()
 .orderBy("count", ascending=____) # False
)

display(customers_by_country)

**Rozkład geograficzny klientów**

Analiza pokazuje dystrybucję klientów według krajów, posortowaną malejąco. 
To pozwala zidentyfikować:
- **Główne rynki**: Kraje z największą bazą klientów 
- **Potencjał ekspansji**: Kraje z małą liczbą klientów
- **Koncentrację geograficzną**: Czy firma ma globalny zasięg

In [0]:
# Sprawdź wartości NULL w każdej kolumnie
from pyspark.sql.functions import col, sum as spark_sum

null_counts = customers_df_manual.select([
 spark_sum(col(c).____().____("int")).alias(c) # isNull, cast
 for c in customers_df_manual.columns
])

display(null_counts)

**Analiza wartości NULL**

Sprawdzenie brakujących danych w każdej kolumnie:
- **0 = Kompletne dane** w kolumnie
- **>0 = Braki danych** wymagające obsługi

**Istotne dla jakości danych**: Braki w kluczowych polach (`customer_id`, `email`) są krytyczne dla biznesu.

In [0]:
# Wygeneruj statystyki opisowe
display(customers_df_manual.____()) # describe

**Statystyki opisowe customers**

Metoda `describe()` pokazuje:
- **count**: Liczba niepustych wartości
- **mean/stddev**: Średnia i odchylenie standardowe (dla kolumn numerycznych)
- **min/max**: Wartości skrajne
- Dla kolumn string: najczęstsze wartości

**Użycie**: Identyfikacja odstających wartości i ogólnego rozkładu danych

---

## Zadanie 5: Eksploracja danych - Orders (15 min)

### Cel:
Przeprowadź analizę zamówień.

### Instrukcje:
1. Policz liczbę zamówień według statusu
2. Oblicz całkowitą wartość zamówień
3. Znajdź średnią, min, max wartość zamówienia
4. Sprawdź braki danych
5. Znajdź 10 najdroższych zamówień

### Oczekiwany rezultat:
- Statystyki biznesowe zamówień
- Identyfikacja problemów z danymi
- Top 10 zamówień

### Wskazówki:
- Użyj `.agg()` z funkcjami: `sum`, `avg`, `min`, `max`
- Do sortowania: `.orderBy(col("column").desc())`

In [0]:
# Policz zamówienia według metod płatności 
orders_by_payment = (
 orders_df_manual
 .groupBy("____") # payment_method
 .____() # count
 .orderBy("count", ascending=False)
)

display(orders_by_payment)

**Rozkład metod płatności**

Analiza pokazuje preferencje płatnicze klientów:
- **Cash, Credit Card, Debit Card, PayPal**: Główne metody
- **Biznesowe wnioski**: Które metody są najpopularniejsze?
- **Planowanie**: Czy potrzebne są dodatkowe metody płatności?

In [0]:
# Oblicz statystyki dla total_amount
from pyspark.sql.functions import sum, avg, min, max, count, round as spark_round

orders_stats = orders_df_manual.select(
 count("*").alias("total_orders"),
 spark_round(____("total_amount"), 2).alias("total_revenue"), # sum
 spark_round(____("total_amount"), 2).alias("avg_order_value"), # avg
 ____("total_amount").alias("min_order"), # min
 ____("total_amount").alias("max_order") # max
)

display(orders_stats)

**Kluczowe metryki biznesowe orders**

Obliczone statystyki pokazują:
- **Total Orders**: Całkowita liczba zamówień
- **Total Revenue**: Suma wszystkich zamówień (przychód)
- **Average Order Value (AOV)**: Średnia wartość zamówienia
- **Min/Max Order**: Zakres wartości zamówień

**Biznesowe zastosowanie**: Benchmarking, planowanie budżetu, analiza trendów

In [0]:
# Znajdź 10 najdroższych zamówień
top_orders = (
 orders_df_manual
 .orderBy(col("total_amount").____()) # desc
 .limit(____) # 10
)

display(top_orders)

**Top 10 najdroższych zamówień**

Analiza największych transakcji pokazuje:
- **VIP customers**: Kto generuje najwyższe przychody?
- **Produkty premium**: Które produkty w najdroższych zamówieniach?
- **Wzorce**: Czy wysokie zamówienia mają wspólne cechy?

**Zastosowanie biznesowe**: Segmentacja klientów, strategie retention, cross-selling

In [0]:
# Sprawdź braki danych w orders
null_counts_orders = orders_df_manual.select([
 spark_sum(col(c).isNull().cast("int")).alias(c) 
 for c in orders_df_manual.columns
])

display(null_counts_orders)

**Analiza braków danych w orders**

Kluczowe braki do sprawdzenia:
- **order_id NULL**: Zamówienia bez identyfikatora (problem systemu?)
- **order_datetime NULL**: Brak czasu transakcji (wpływ na reporting)
- **customer_id NULL**: Niemożliwość powiązania z klientem

**Akcje naprawcze**: Usunięcie lub wypełnienie brakujących wartości w kolejnych krokach ETL

---

## Zadanie 6: Eksploracja danych - Products (10 min)

### Cel:
Przeprowadź analizę produktów.

### Instrukcje:
1. Sprawdź schemat i kolumny
2. Policz produkty według kategorii (jeśli kolumna istnieje)
3. Znajdź statystyki cenowe (jeśli kolumna price istnieje)
4. Wyświetl 10 najdroższych produktów

### Oczekiwany rezultat:
- Kompletna analiza produktów
- Rozkład kategorii
- Statystyki cenowe

### Wskazówki:
- Sprawdź dostępne kolumny przed analizą: `products_df.columns`

In [0]:
# Sprawdź schemat i kolumny
products_df.columns
products_df.printSchema()

# Wyświetl przykładowe dane
display(products_df.limit(5))

**Analiza struktury products**

Sprawdzenie schematu pozwala zrozumieć:
- **Dostępne kolumny**: Jakie informacje o produktach są dostępne?
- **Typy danych**: Parquet automatycznie zachowuje właściwe typy
- **Przykładowe dane**: Zawartość i format wartości

**Następne kroki**: Sprawdź czy istnieją kolumny `category` i `price` do dalszej analizy

In [0]:
# Sprawdź czy kolumna 'category' istnieje, policz produkty według kategorii
if "category" in products_df.columns:
 products_by_category = (
 products_df
 .groupBy("____") # category
 .count()
 .orderBy("____", ascending=False) # count
 )
 
 display(products_by_category)
else:
 display(spark.createDataFrame([("Kolumna 'category' nie istnieje w danych",)], ["info"]))

**Rozkład kategorii produktów**

Jeśli kolumna `category` istnieje:
- **Portfel produktowy**: Jakie kategorie są dostępne?
- **Koncentracja**: Które kategorie dominują w ofercie?
- **Możliwości cross-selling**: Powiązane kategorie

Jeśli nie istnieje - może być potrzebne wzbogacenie danych o kategorizację produktów.

In [0]:
# Sprawdź czy kolumna 'price' istnieje, oblicz statystyki
if "price" in products_df.columns:
 products_stats = products_df.select(
 ____("*").alias("total_products"), # count
 spark_round(avg("____"), 2).alias("avg_price"), # price
 min("price").alias("min_price"),
 ____("price").alias("max_price") # max
 )
 
 display(products_stats)
else:
 display(spark.createDataFrame([("Kolumna 'price' nie istnieje w danych",)], ["info"]))

**Statystyki cenowe produktów**

Jeśli kolumna `price` istnieje, analizujemy:
- **Średnia cena**: Ogólny poziom cenowy
- **Zakres cen**: min/max dla zrozumienia portfela
- **Segmentacja**: Produkty tanie vs premium

**Zastosowanie**: Pricing strategy, analiza konkurencyjności, segmentacja produktów

---

## Zadanie 7: Analiza jakości danych (15 min)

### Cel:
Stwórz raport jakości danych dla wszystkich trzech datasetów.

### Instrukcje:
1. Dla każdego datasetu policz:
 - Liczbę rekordów
 - Liczbę kolumn
 - Liczbę wartości NULL w każdej kolumnie
 - Procent kompletności danych
2. Zidentyfikuj duplikaty w każdym datasecie
3. Znajdź potencjalne problemy (np. negatywne wartości, nietypowe daty)

### Oczekiwany rezultat:
- Kompletny raport jakości danych
- Lista zidentyfikowanych problemów
- Rekomendacje działań naprawczych

### Wskazówki:
- Użyj `.count()`, `.distinct().count()` do wykrywania duplikatów
- Użyj filtrów do znajdowania nietypowych wartości

In [0]:
# Funkcja do analizy jakości danych
def data_quality_report(df, dataset_name):
 total_rows = df.count()
 total_cols = len(df.columns)
 
 # Sprawdź duplikaty
 distinct_rows = df.____().count() # distinct
 duplicates = total_rows - distinct_rows
 dup_pct = (duplicates / total_rows * 100) if total_rows > 0 else 0
 
 # Sprawdź wartości NULL
 null_counts = df.select([
 spark_sum(col(c).isNull().cast("int")).alias(c) 
 for c in df.columns
 ]).collect()[0].asDict()
 
 # Stwórz podsumowanie
 summary_data = [
 (dataset_name, total_rows, total_cols, ____, # duplicates
 f"{dup_pct:.2f}%", sum(null_counts.values()))
 ]
 
 summary_df = spark.createDataFrame(summary_data, 
 ["Dataset", "Total_Rows", "Total_Cols", "Duplicates", "Dup_Percent", "Total_NULLs"])
 
 return summary_df, null_counts

# Uruchom raport dla każdego datasetu
customers_summary, customers_nulls = data_quality_report(____, "Customers") # customers_df_manual
orders_summary, orders_nulls = data_quality_report(orders_df_manual, "Orders")
products_summary, products_nulls = data_quality_report(____, "Products") # products_df

# Połącz wszystkie podsumowania
combined_summary = customers_summary.union(orders_summary).union(products_summary)
display(combined_summary)

**Skonsolidowany raport jakości danych**

Podsumowanie pokazuje dla każdego datasetu:
- **Total_Rows**: Liczba rekordów
- **Total_Cols**: Liczba kolumn 
- **Duplicates**: Liczba duplikatów
- **Dup_Percent**: Procent duplikatów
- **Total_NULLs**: Suma wszystkich wartości NULL

**Kluczowe pytania:**
- Które datasety mają najwięcej problemów jakości?
- Czy duplikaty są akceptowalne biznesowo?
- Które kolumny wymagają wypełnienia braków?

In [0]:
# Sprawdź problematyczne wartości w orders
negative_orders = orders_df_manual.filter(col("____") < 0).count() # total_amount
zero_orders = orders_df_manual.filter(col("total_amount") == ____).count() # 0

# Sprawdź nietypowe daty
from pyspark.sql.functions import year, current_date, datediff

future_orders = orders_df_manual.filter(col("order_datetime") > ____()).count() # current_date
old_orders = orders_df_manual.filter(year(col("____")) < 2020).count() # order_datetime

# Stwórz podsumowanie problemów
problems_data = [
 ("Negatywne wartości zamówień", negative_orders),
 ("Zamówienia z wartością zero", zero_orders),
 ("Zamówienia z przyszłą datą", future_orders),
 ("Zamówienia starsze niż 2020", old_orders)
]

problems_df = spark.createDataFrame(problems_data, ["Problem", "Count"])
display(problems_df)

**Identyfikacja problemów jakości danych**

Znalezione anomalie w danych:
- **Negatywne wartości**: Błędy systemowe lub zwroty?
- **Wartości zero**: Promocje czy błędy?
- **Przyszłe daty**: Błędy wprowadzania danych
- **Bardzo stare daty**: Dane testowe czy prawdziwe?

**Następne kroki**: Decyzja biznesowa - usunąć, poprawić czy oznacza jako odstające?

---

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] Wszystkie 3 datasety wczytane (customers, orders, products)
- [ ] Schematy zdefiniowane manualnie dla CSV i JSON
- [ ] Kompletna eksploracja każdego datasetu
- [ ] Raport jakości danych wygenerowany
- [ ] Zidentyfikowane problemy z danymi

### Komendy weryfikacyjne:

In [0]:
print("="*60)
print("=== WERYFIKACJA WYNIKÓW WARSZTATU ===")
print("="*60)

# Weryfikacja wyników warsztatu
try:
 assert 'customers_df_manual' in locals(), "customers_df_manual nie został utworzony"
 assert 'orders_df_manual' in locals(), "orders_df_manual nie został utworzony"
 assert 'products_df' in locals(), "products_df nie został utworzony"
 
 # Sprawdź liczby rekordów
 verification_data = [
 ("Customers", customers_df_manual.count()),
 ("Orders", orders_df_manual.count()),
 ("Products", products_df.count())
 ]
 
 verification_df = spark.createDataFrame(verification_data, ["Dataset", "Record_Count"])
 display(verification_df)
 
 # Sprawdź tabele Delta Lake
 tables_to_check = [
 f"{BRONZE_SCHEMA}.customers_workshop",
 f"{BRONZE_SCHEMA}.orders_workshop", 
 f"{BRONZE_SCHEMA}.products_workshop"
 ]
 
 table_status = []
 for table in tables_to_check:
 try:
 spark.table(table)
 table_status.append((table, " Exists"))
 except:
 table_status.append((table, "✗ Missing"))
 
 tables_df = spark.createDataFrame(table_status, ["Table", "Status"])
 display(tables_df)
 
 print(" WARSZTAT ZAKOŃCZONY POMYŚLNIE! ")
 
except (AssertionError, NameError) as e:
 print(f"✗ Błąd: {e}")
 print("Sprawdź czy wszystkie komórki zostały wykonane poprawnie.")

**Wyniki weryfikacji warsztatu**

Tabela weryfikacji pokazuje:
- **Record_Count**: Liczba rekordów w każdym DataFrame
- **Table Status**: Czy tabele Delta Lake zostały utworzone

**Warunki sukcesu:**
- Wszystkie DataFrames utworzone
- Liczba rekordów > 0
- Wszystkie tabele zapisane w Delta Lake

---

## Troubleshooting

### Problem 1: Błąd "Path does not exist"
**Objawy:**
- Błąd podczas wczytywania plików CSV/JSON/Parquet
- Komunikat: `Path does not exist: /Volumes/...`

**Rozwiązanie:**
```python
# Sprawdź, czy ścieżka istnieje
dbutils.fs.ls(DATASET_BASE_PATH)

# Upewnij się, że zmienne są poprawnie zdefiniowane
print(f"DATASET_BASE_PATH: {DATASET_BASE_PATH}")
print(f"CUSTOMERS_CSV: {CUSTOMERS_CSV}")
```

### Problem 2: Błąd typu danych podczas wczytywania
**Objawy:**
- Wartości NULL w miejscach, gdzie powinny być dane
- Niepoprawne typy kolumn (wszystko jako string)

**Rozwiązanie:**
- Zawsze definiuj schemat manualnie zamiast używać `inferSchema=True`
- Użyj odpowiednich typów: `IntegerType()`, `StringType()`, `TimestampType()`, `DoubleType()`

### Problem 3: Tabela już istnieje
**Objawy:**
- Błąd: `Table already exists`

**Rozwiązanie:**
```python
# Użyj mode="overwrite"
df.write.format("delta").mode("overwrite").saveAsTable(table_name)

# Lub usuń tabelę przed zapisem
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
```

### Debugging tips:
- Użyj `.printSchema()` aby sprawdzić strukturę DataFrame
- Użyj `.explain()` aby zobaczyć plan wykonania
- Sprawdź liczbę rekordów: `df.count()`
- Wyświetl przykładowe dane: `display(df.limit(10))`

## Best Practices

### Wydajność:
- **Zawsze definiuj schemat manualnie** - `inferSchema=True` jest wolne i może dawać błędne typy
- **Używaj Parquet dla dużych datasetów** - najlepsza kompresja i wydajność
- **Partycjonuj duże tabele** według często używanych kolumn filtrujących
- **Używaj `.cache()` tylko gdy DataFrame jest wielokrotnie użyty**

### Jakość kodu:
- **Nazywaj DataFrames opisowo**: `customers_df`, `orders_clean_df`, `products_enriched_df`
- **Dodawaj komentarze** wyjaśniające logikę biznesową
- **Sprawdzaj dane po każdym kroku** transformacji
- **Używaj zmiennych dla nazw tabel i schematów** (nie hardcoduj)

### Data Quality:
- **Zawsze sprawdzaj wartości NULL** przed transformacjami
- **Wykrywaj duplikaty**: `df.distinct().count()` vs `df.count()`
- **Waliduj zakresy wartości**: negatywne ceny, przyszłe daty
- **Dokumentuj znalezione problemy** w komentarzach lub osobnym notebooku

### Governance:
- **Używaj Unity Catalog** dla zarządzania tabelami i uprawnieniami
- **Stosuj konwencję nazewnictwa**: `bronze.customers`, `silver.customers_clean`, `gold.customers_agg`
- **Separuj środowiska**: dev/staging/prod używając różnych katalogów
- **Loguj operacje**: zapisuj metadane o przetwarzanych danych

---

## Podsumowanie

### Co zostało osiągnięte:
- Skonfigurowałeś środowisko Databricks i zmienne per-user
- Wczytałeś dane z trzech różnych formatów (CSV, JSON, Parquet)
- Zdefiniowałeś schematy manualnie (best practice)
- Przeprowadziłeś szczegółową eksplorację wszystkich datasetów
- Zidentyfikowałeś problemy jakości danych
- Wygenerowałeś kompletny raport jakości danych
- Zapisałeś dane do tabel Delta Lake

### Kluczowe wnioski:
1. **Ręczne schematy > inferSchema**: Szybsze, bezpieczniejsze i przewidywalne typy danych
2. **Eksploracja przed transformacją**: Zawsze analizuj dane przed rozpoczęciem przetwarzania
3. **Jakość wymaga monitorowania**: NULL, duplikaty, outliers muszą być wykrywane systematycznie
4. **Parquet to standard lakehouse**: Wbudowany schemat, najlepsza kompresja i wydajność

### Quick Reference - Najważniejsze komendy:

| Operacja | PySpark | Notatki |
|----------|---------|---------|
| Wczytaj CSV | `spark.read.format("csv").option("header","true").schema(schema).load(path)` | Zawsze używaj ręcznego schematu |
| Wczytaj JSON | `spark.read.format("json").schema(schema).load(path)` | Opcja `multiLine=true` dla JSON arrays |
| Wczytaj Parquet | `spark.read.format("parquet").load(path)` | Schemat wbudowany |
| Zapisz do Delta | `df.write.format("delta").mode("overwrite").saveAsTable(table)` | mode: overwrite/append |
| Sprawdź NULL | `df.select([sum(col(c).isNull()).alias(c) for c in df.columns])` | Dla każdej kolumny |
| Znajdź duplikaty | `df.count() - df.distinct().count()` | Różnica = liczba duplikatów |
| Statystyki | `df.describe()` | Dla kolumn numerycznych |
| Grupowanie | `df.groupBy("col").count()` | Agregacje |

### Następne kroki:
- **Kolejny warsztat**: `02_transformations_cleaning_workshop.ipynb` - Transformacje i czyszczenie danych
- **Praktyka**: Powtórz eksplorację na własnych datasetach
- **Dokumentacja**: [Databricks Data Import Best Practices](https://docs.databricks.com/ingestion/index.html)
- **Zadanie domowe**: Wykonaj analizę eksploracyjną dodatkowego datasetu (np. transactions, events)

## Czyszczenie zasobów

Opcjonalne - usuń tabele utworzone podczas warsztatu:

In [0]:
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone tabele
# Te tabele mogą być potrzebne w kolejnych warsztatach!

# Odkomentuj poniższe linie aby usunąć tabele:
# spark.sql(f"DROP TABLE IF EXISTS {BRONZE_SCHEMA}.customers_workshop")
# spark.sql(f"DROP TABLE IF EXISTS {BRONZE_SCHEMA}.orders_workshop") 
# spark.sql(f"DROP TABLE IF EXISTS {BRONZE_SCHEMA}.products_workshop")
# spark.catalog.clearCache()

display(spark.createDataFrame([
 ("Czyszczenie zasobów jest zakomentowane",),
 ("Odkomentuj kod powyżej aby usunąć tabele",),
 ("UWAGA: Tabele mogą być potrzebne w kolejnych warsztatach!",)
], ["Info"]))